<a href="https://colab.research.google.com/github/onuralpArsln/MlAiTutorialProjects/blob/main/8-Generative/dcganAdaptive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Convolutional Generative Adversarial Network owndata

DCGAN   iki parçalı bir sistemdir. Generator ve Discriminator eğitilir. Böylece amaca uygun bir görüntü oluşturmaya çalışan Generator ve onu yakalmaaya çalışan Discriminator oluşur. Ne zaman discriminator kandırlırsa o zaman sonuç verilir bu yüzden adverserial sayılır.

### Setup


Tensorflow ve diğer gereksinimler

In [3]:
# Bu sonrasında şov amaçlı ara parçaları görmek için epoch sonuçlarından gif yapacağız.
!pip install imageio -q
!pip install git+https://github.com/tensorflow/docs -q

In [4]:
import tensorflow as tf
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from IPython import display
tf.__version__

### Load and prepare the dataset




Eğer kendi datasetinde çalışmak istersen

In [5]:
!wget -q  https://github.com/onuralpArsln/MlAiTutorialProjects/raw/refs/heads/main/8-Generative/db64miniset.zip

In [6]:
!unzip -qq db64miniset.zip

!wget -q  https://github.com/onuralpArsln/MlAiTutorialProjects/raw/refs/heads/main/8-Generative/db32set6k.zip

In [ ]:
!unzip -qq db32set6k.zip

In [45]:
##set your data size input
img_size=64

In [8]:
# Dataset yükleme fonksiyonu
def load_custom_images(image_dir, img_size=(img_size, img_size)):
    image_paths = glob.glob(image_dir + "/*.png")  # Adjust extension if needed
    images = [img_to_array(load_img(img, color_mode="grayscale", target_size=img_size)) for img in image_paths]
    images = np.array(images).astype("float32") / 127.5 - 1  # Normalize to [-1, 1]
    return images

# Replace dataset loading
image_dir = "db64miniset"
train_images = load_custom_images(image_dir)

# Ensure correct shape
train_images = train_images.reshape(train_images.shape[0], img_size, img_size, 1)

# Create TensorFlow dataset
BUFFER_SIZE = len(train_images) # kaç adet görsel var
BATCH_SIZE = 256
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)


## Create the models

Generator ve discriminator [Keras Sequential API](https://www.tensorflow.org/guide/keras#sequential_model) Kullanır.

### The Generator

The generator uses `tf.keras.layers.Conv2DTranspose` (upsampling) layers to produce an image from a seed (random noise). Start with a `Dense` layer that takes this seed as input, then upsample several times until you reach the desired image size of 28x28x1. Notice the `tf.keras.layers.LeakyReLU` activation for each layer, except the output layer which uses tanh.

In [46]:
def make_generator_model():
    """
    Creates the generator model that transforms random noise into images.
    The generator "learns" to create fake images that look real.
    
    Key Changes from Original:
    - Better scaling of layers based on image size
    - Added more filters in early layers for better feature learning
    - Improved layer progression for stability
    """
    global img_size
    # Make sure image size is valid (must be divisible by 16 because we have 4 upsampling layers)
    assert img_size % 16 == 0, "Image size must be divisible by 16"
    
    # Calculate the initial size for our first layer
    # If img_size is 64, initial_size will be 4 (64/16)
    initial_size = img_size // 16
    
    model = tf.keras.Sequential([
        # Input layer - takes random noise vector of size 100
        layers.Input(shape=(100,)),
        
        # First layer: Dense layer to create enough values to reshape into a small image
        # If img_size=64, this creates a 4x4x512 tensor from our random input
        layers.Dense(initial_size * initial_size * 512),
        layers.BatchNormalization(),  # Stabilizes training by normalizing layer outputs
        layers.LeakyReLU(alpha=0.2),  # Better than regular ReLU for GANs - allows small negative values
        layers.Reshape((initial_size, initial_size, 512)),
        
        # Upsampling Block 1: 4x4 -> 8x8
        # Conv2DTranspose is like a reverse convolution - it makes the image bigger
        layers.Conv2DTranspose(256, 5, strides=2, padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        
        # Upsampling Block 2: 8x8 -> 16x16
        layers.Conv2DTranspose(128, 5, strides=2, padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        
        # Upsampling Block 3: 16x16 -> 32x32
        layers.Conv2DTranspose(64, 5, strides=2, padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        
        # Upsampling Block 4: 32x32 -> 64x64
        layers.Conv2DTranspose(32, 5, strides=2, padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        
        # Final layer: Creates the actual image with pixel values between -1 and 1
        layers.Conv2D(1, 5, padding='same', activation='tanh')
    ])
    
    return model


Use the (as yet untrained) generator to create an image.

In [47]:
generator = make_generator_model()

noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)

plt.imshow(generated_image[0, :, :, 0], cmap='gray')

### The Discriminator

Discriminator CNN mantığı ile çalışsan bir classification yapısıdır.

In [48]:
def make_discriminator_model():
    """
    Creates the discriminator model that tries to tell real images from fake ones.
    Think of it as a detective trying to spot forgeries.
    
    Key Changes from Original:
    - Added more dropout to prevent overfitting
    - Better layer progression
    - Improved filter counts for better feature detection
    """
    global img_size
    
    model = tf.keras.Sequential([
        # Input layer: Takes an image of our specified size
        layers.Input(shape=(img_size, img_size, 1)),
        
        # Block 1: Shrink image and extract basic features
        # If input is 64x64, this makes it 32x32
        layers.Conv2D(64, 5, strides=2, padding='same'),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.3),  # Prevents overfitting by randomly turning off 30% of neurons
        
        # Block 2: 32x32 -> 16x16
        # Doubling filters each time to maintain information density
        layers.Conv2D(128, 5, strides=2, padding='same'),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.3),
        
        # Block 3: 16x16 -> 8x8
        layers.Conv2D(256, 5, strides=2, padding='same'),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.3),
        
        # Block 4: 8x8 -> 4x4
        layers.Conv2D(512, 5, strides=2, padding='same'),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.3),
        
        # Final layers: Flatten the 3D tensor to 1D and output a single value
        # This value determines if the discriminator thinks the image is real or fake
        layers.Flatten(),
        layers.Dense(1)  # No activation = linear activation
    ])
    
    return model

Use the (as yet untrained) discriminator to classify the generated images as real or fake. The model will be trained to output positive values for real images, and negative values for fake images.

In [49]:
generated_image = generator(tf.random.normal([1, 100]), training=False)
print(generated_image.shape)  # Should be (1, img_size, img_size, 1)

In [50]:
discriminator = make_discriminator_model()
decision = discriminator(generated_image)
print (decision)

## Define the loss and optimizers

Define loss functions and optimizers for both models.


In [51]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

### Discriminator loss

This method quantifies how well the discriminator is able to distinguish real images from fakes. It compares the discriminator's predictions on real images to an array of 1s, and the discriminator's predictions on fake (generated) images to an array of 0s.

In [ ]:
def discriminator_loss(real_output, fake_output):
    """
    Calculates how well the discriminator can distinguish real from fake images.
    Lower loss = better at telling them apart.
    """
    cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    # Real images should be classified as 1 (real)
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    # Fake images should be classified as 0 (fake)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    # Total loss is the sum of both
    total_loss = real_loss + fake_loss
    return total_loss

### Generator loss
The generator's loss quantifies how well it was able to trick the discriminator. Intuitively, if the generator is performing well, the discriminator will classify the fake images as real (or 1). Here, compare the discriminators decisions on the generated images to an array of 1s.

In [ ]:
def generator_loss(fake_output):
    """
    Calculates how well the generator fooled the discriminator.
    Lower loss = better at creating convincing images.
    """
    cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    # Generator wants discriminator to think its images are real (1)
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:

# New addition: Custom learning rate scheduler for better training
class CustomLearningRateScheduler:
    """
    Controls how fast the model learns over time.
    Gradually reduces learning rate to fine-tune the learning process.
    """
    def __init__(self, initial_lr=1e-4, decay_factor=0.95, decay_epochs=10):
        self.initial_lr = initial_lr
        self.decay_factor = decay_factor
        self.decay_epochs = decay_epochs
        
    def get_learning_rate(self, epoch):
        # Reduces learning rate over time for more stable training
        return self.initial_lr * (self.decay_factor ** (epoch // self.decay_epochs))

The discriminator and the generator optimizers are different since you will train two networks separately.

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

### Save checkpoints
This notebook also demonstrates how to save and restore models, which can be helpful in case a long running training task is interrupted.

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

## Define the training loop


In [ ]:
EPOCHS = 100
noise_dim = 100
num_examples_to_generate = 16

# You will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

The training loop begins with generator receiving a random seed as input. That seed is used to produce an image. The discriminator is then used to classify real images (drawn from the training set) and fakes images (produced by the generator). The loss is calculated for each of these models, and the gradients are used to update the generator and discriminator.

In [ ]:
@tf.function  # Makes training faster by compiling the function
def train_step(images, generator, discriminator, generator_optimizer, discriminator_optimizer, batch_size=256):
    """
    Performs a single training step on both generator and discriminator.
    
    Changes from original:
    - Better gradient handling
    - More stable training process
    - Improved batch processing
    """
    noise_dim = 100
    # Create random noise for generator input
    noise = tf.random.normal([batch_size, noise_dim])
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # Generate fake images
        generated_images = generator(noise, training=True)
        
        # Get discriminator's opinion on both real and fake images
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)
        
        # Calculate losses
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)
    
    # Calculate and apply gradients for both networks
    gen_gradients = gen_tape.gradient(gen_loss, generator.trainable_variables)
    disc_gradients = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    
    generator_optimizer.apply_gradients(zip(gen_gradients, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(disc_gradients, discriminator.trainable_variables))
    
    return gen_loss, disc_loss

In [ ]:
#old train 
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for image_batch in dataset:
      train_step(image_batch)

    # Produce images for the GIF as you go
    display.clear_output(wait=True)
    generate_and_save_images(generator,
                             epoch + 1,
                             seed)

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Generate after the final epoch
  display.clear_output(wait=True)
  generate_and_save_images(generator,
                           epochs,
                           seed)

In [ ]:
## new train
def train(dataset, epochs, batch_size=256):
    """
    Main training loop.
    
    Changes from original:
    - Added learning rate scheduling
    - Better progress monitoring
    - Improved optimization parameters
    """
    generator = make_generator_model()
    discriminator = make_discriminator_model()
    
    # Initialize optimizers with learning rate scheduler
    lr_scheduler = CustomLearningRateScheduler()
    # Beta1=0.5 is a common choice for GANs - helps with training stability
    generator_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_scheduler.initial_lr, beta_1=0.5)
    discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_scheduler.initial_lr, beta_1=0.5)
    
    for epoch in range(epochs):
        # Update learning rates each epoch
        lr = lr_scheduler.get_learning_rate(epoch)
        generator_optimizer.learning_rate = lr
        discriminator_optimizer.learning_rate = lr
        
        for batch in dataset:
            gen_loss, disc_loss = train_step(batch, generator, discriminator, 
                                           generator_optimizer, discriminator_optimizer,
                                           batch_size)
            
        # Print progress every 10 epochs
        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1}, Gen Loss: {gen_loss:.4f}, Disc Loss: {disc_loss:.4f}')
    
    return generator, discriminator

**Generate and save images**


In [ ]:
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)

  fig = plt.figure(figsize=(4, 4))

  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
      plt.axis('off')

  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

## Train the model
Call the `train()` method defined above to train the generator and discriminator simultaneously. Note, training GANs can be tricky. It's important that the generator and discriminator do not overpower each other (e.g., that they train at a similar rate).

At the beginning of the training, the generated images look like random noise. As training progresses, the generated digits will look increasingly real. After about 50 epochs, they resemble MNIST digits. This may take about one minute / epoch with the default settings on Colab.

In [ ]:
train(train_dataset, EPOCHS)

Restore the latest checkpoint.

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

## Create a GIF


In [ ]:
# Display a single image using the epoch number
def display_image(epoch_no):
  return PIL.Image.open('image_at_epoch_{:04d}.png'.format(epoch_no))

In [ ]:
display_image(EPOCHS)

Use `imageio` to create an animated gif using the images saved during training.

In [ ]:
anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('image*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

In [ ]:
import tensorflow_docs.vis.embed as embed
embed.embed_file(anim_file)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generate random noise (latent vector)
latent_dim = 100  # Same as used in training
random_noise = tf.random.normal([1, latent_dim])

# Generate an image
generated_image = generator(random_noise, training=False)

# Rescale pixel values from [-1, 1] to [0, 1]
generated_image = (generated_image + 1) / 2.0

# Display the image
plt.imshow(generated_image[0, :, :, 0], cmap="gray")  # Use cmap="gray" for grayscale
plt.axis("off")
plt.show()
